<a href="https://colab.research.google.com/github/EllieKallmier/sola5050-t1-2025/blob/main/Bill_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tariff Calculator

Includes:
* Bill calculation for load profiles with and without PV
* Calculates flat rate, time of use and demand charges and includes feed in credits and daily usage charges.
* Option to add a battery with a simple algorithm to maximise self-consumption.